### Training a LDA model on sample text and visualisation

In [ ]:
fname = '/Users/mz/Desktop/text.txt'
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 
doc_complete = content

word frequency no need to use

In [4]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(doc_complete)
freq = np.ravel(X.sum(axis=0))

In [5]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

Define stopwords and normalize corpus

In [6]:
exclude = set(string.punctuation) 
stop = set(stopwords.words('english')+['et','al,','al.,','al'])
lemma = WordNetLemmatizer()


clean document from stopwords and puctuations

In [7]:
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [8]:
doc_clean = [clean(doc).split() for doc in doc_complete]   


#remove high or low frequency words


In [9]:
high_fre = 25; low_fre = 1
all_tokens = sum(doc_clean, [])
tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) <= low_fre or all_tokens.count(word)>=high_fre)
doc_clean = [[word for word in text if word not in tokens_once]for text in doc_clean]

In [10]:
# Importing Gensim
import gensim
from gensim import corpora

Using TensorFlow backend.


In [12]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)
print(dictionary)


Dictionary(69 unique tokens: ['small', 'participant', 'first', 'klingberg', 'subprocesses']...)


In [13]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [14]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

In [ ]:
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes=100)

print(ldamodel.print_topics(num_topics=2, num_words=3))

#### LDA visualisation

In [20]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
pyLDAvis.display(vis)


Does word-word cooccurence matrix

In [ ]:
def co_mat(doc):
    from sklearn.feature_extraction.text import CountVectorizer
    import numpy as np
    count_model = CountVectorizer(ngram_range=(1,1)) # default unigram model
    X = count_model.fit_transform(doc)
    Xc = (X.T * X) # this is co-occurrence matrix in sparse csr format
    Xc.setdiag(0) # sometimes you want to fill same word cooccurence to 0
    print(Xc.todense()) # print out matrix in dense format
    return np.asarray(Xc.todense()), count_model.vocabulary_

mat, mat_dictionary = co_mat(new_doc)